In [57]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch import get_file_path
from torch.utils.data import Dataset, DataLoader
import os

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3070


In [59]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_dir)

    def __getitem__(self, idx):
        img_path = self.img_dir[idx]
        image = Image.open(img_path)

        label = int(os.path.basename(img_path).split('_')[0])

        if self.transform:
            image = self.transform(image)

        return image, label

In [60]:
def get_file_paths(folder):
    return [os.path.join(folder, fname) for fname in os.listdir(folder) if fname.endswith('.jpg')]


train_data = get_file_paths('dataset/training')
val_data = get_file_paths('dataset/validation')
eval_data = get_file_paths('dataset/evaluation')

In [61]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [62]:
train_dataset = CustomImageDataset(train_data, transform=transform)
val_dataset = CustomImageDataset(val_data, transform=transform)
eval_dataset = CustomImageDataset(eval_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=True)

In [63]:
image, label = train_dataset[0]

In [64]:
image.size()

torch.Size([3, 32, 32])

In [65]:
class_names = ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles/Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable/Fruit']

In [66]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)  # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2)  # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 4)  # (24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 11)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [67]:
net = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [68]:
for epoch in range(10):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.3627
Training epoch 1...
Loss: 2.2798
Training epoch 2...
Loss: 2.2568
Training epoch 3...
Loss: 2.2373
Training epoch 4...
Loss: 2.2183
Training epoch 5...
Loss: 2.1984
Training epoch 6...
Loss: 2.1786
Training epoch 7...
Loss: 2.1511
Training epoch 8...
Loss: 2.1132
Training epoch 9...
Loss: 2.0860


In [69]:
torch.save(net.state_dict(), 'trained_nett.pth')

In [70]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_nett.pth'))

C:\Users\Ole-Johan\AppData\Local\Temp\ipykernel_11984\775912913.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_nett.pth'))


<All keys matched successfully>

In [71]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in eval_loader:
        images, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!